<div>
<img src="https://raw.githubusercontent.com/RafaelCaballero/tdm/refs/heads/master/images/mioti.png" width=600 />
</ivdi>
<div style="background-color: #E9967A; color: black; padding: 10px; border-radius: 5px;">
<h1>Proyecto Final: Estudio sobre la quema de calorías en el gimnasio </h1>
</div>

# Tabla de contenidos
1. [Carga de datos, limpieza y reestructuración](#carga-de-datos,-limpieza-y-reestructuracion)<br>
2. [Graficación de las variables de manera individual](#graficacion-de-las-variables-de-manera-individual)<br>
3. [Outliers](#outliers)<br>
4. [Test estadísticos](#test-estadisticos)<br>
5. [Correlaciones entre variables](#correlaciones-entre-variables)<br>
6. [Modelo de regresión](#modelo-de-regresion)<br>
7. [Conclusiones](#conclusiones)<br>

## CARGA DE DATOS, LIMPIEZA Y REESTRUCTURACION

Primeramente, leemos los datos del csv.

In [ ]:
#Carga de datos
import numpy as np
import pandas as pd

df= pd.read_csv('gym_members_exercise_tracking.csv', sep=',')
df.head(10)

Eliminamos la columna 'water_intake' ya que no nos interesa.

In [ ]:
df= df.drop(columns='Water_Intake (liters)')

Obtenemos, con funciones predeterminadas de pandas, algunas estadísticas para ir familiarizándonos con los datos.

In [ ]:
df.describe()

In [ ]:
def info_df(df):
    return pd.DataFrame({
        'Columna': df.columns,
        'No Nulos': df.notnull().sum().values,
        'Nulos': df.isnull().sum().values,
        'Tipo Python': df.dtypes.values,
        'Núm. valores': [ df[col].cat.ordered  if isinstance(df[col].dtype, pd.CategoricalDtype) else None
            for col in df.columns  ]
    })

for col in df.columns:
    if len(df[col].unique())<10:
        print(df[col].value_counts())
        print("="*50)
        
info_df(df)

Convertimos las columnas 'Gender', 'Workout_Type', 'Workout_Frequency' y 'Experience_Level' en variables categóricas:

In [ ]:
from pandas.api.types import CategoricalDtype 

cat_type=CategoricalDtype(categories=['Male','Female'], ordered=False)
df['Gender'] = df['Gender'].astype(cat_type)

cat_type = CategoricalDtype(categories=['Strength','Cardio','Yoga','HIIT'], ordered=False) 
df['Workout_Type'] = df['Workout_Type'].astype(cat_type)

df['Workout_Frequency (days/week)']=df['Workout_Frequency (days/week)'].astype(str)
cat_type = CategoricalDtype(categories=['2','3','4','5'], ordered=False) 
df['Workout_Frequency (days/week)'] = df['Workout_Frequency (days/week)'].astype(cat_type)

#La columna 'experience_level' la convertimos en palabras y después cambiamos su tipo
mapa={1:'Principiante',2:'Intermedio',3:'Avanzado'}
df['Experience_Level']=df['Experience_Level'].map(mapa)
cat_type=CategoricalDtype(categories=['Principiante','Intermedio','Avanzado'], ordered=False)
df['Experience_Level']=df['Experience_Level'].astype(cat_type)

info_df(df)

## GRAFICACION DE LAS VARIABLES DE MANERA INDIVIDUAL

<div style="background-color: blue; padding: 10px; border-radius: 5px;">
Ahora pasamos a presentar algunas gráficas  de las variabes de manera individual para entender mejor los datos. No se presentan gráficas de todas las variables por motivo de síntetis.
</div>

Primero graficamos todos los histogramas de las variables continuas:

In [ ]:
#Histogramas de las variables continuas
import matplotlib.pyplot as plt

# Columnas a graficar
columnas_a_graficar = ['Age', 'Weight (kg)', 'Height (m)','Max_BPM', 'Avg_BPM', 'Session_Duration (hours)','Calories_Burned','Fat_Percentage','BMI']

# Crear subgráficos
fig, axes = plt.subplots(3, 3, figsize=(12, 8))
axes = axes.flatten()

for i, col in enumerate(columnas_a_graficar):
    ax = axes[i]
    df[col].plot(kind='density', ax=ax, color='blue', xlim=(df[col].min(), df[col].max()))
    df[col].plot(kind='hist', ax=ax, bins=30, density=True, alpha=0.5, color='orange')
    ax.set_title(f'Distribución de {col}')
    ax.set_xlabel(col)
    ax.set_ylabel('Densidad/Frecuencia')

# Ajustar diseño
plt.tight_layout()
plt.show()


Comentarios de cada una de las gráficas:
- Distribución de Age: se observa una distribución prácticamente uniforme.

- Distribución de Weight (kg): se observa una distribucióon normal sesgada a la derecha (positiva). La mayoría de pesos se concentran entre los 60 y los 80 kg.
- Distribución de Height (m): se observa una distribución más o menos bimodal, con los picos entorno a 1,63 m y a 1,78 m. Es evidente que estos dos picos se corresponden con las estaturas medias de cada uno de los sexos (Masculino y Femenino).
- Distribución de Max_BPM: se observa una distribución uniforme.
- Distribución de Avg_BPM: se observa una distribución uniforme,  con un ligero pico (casi despreciable) en torno a los 132 BPM.
- Distribución de Session_Duration (hours): se observa una distribución más o menos normal simétrica. No obstante, el extremo derecho tiene una ligera mayor preponderancia.
- Distribución de Calories_Burned: se observa una distribución normal sesgada a derecha (positiva). Los valores más comunes son entre 800 y 900.
- Distribución de Fat_Percentage: se observa a grandes rasgos una distribución normal sesgada a izquierda (negativa).
- Distribución de BMI: se observa una distribución normal sesgada a derecha (positiva). Valores más comunes en torno a 25, como cabría esperar. 

A continuación, se muestran todos los gráficos de barras agrupados en una imagen por motivos de síntesis. Las variables aquí mostradas son de tipo categórico y nos dan la siguiente información:

In [ ]:
#Todos los gráficos de barras juntos.
fig, axes= plt.subplots(2,2,figsize=(20,18))

df['Experience_Level'].value_counts().plot(kind='bar',ax=axes[0,0],figsize=(10, 8), color='blue', alpha=0.5, title='Frecuencia de cada nivel de exp.',
                                           xlabel='Nivel de experiencia')
df['Workout_Type'].value_counts().plot(kind='bar',ax=axes[0,1],figsize=(10, 8), color='orange',alpha=0.5, title='Frecuencia de cada tipo de entrenamiento',
                                       xlabel='Tipo de entrenamiento')
df['Gender'].value_counts().plot(kind='bar',ax=axes[1,0],figsize=(10, 8), color='orange',alpha=0.5, title='Frecuencia de género',
                                 xlabel='Género')
df['Workout_Frequency (days/week)'].value_counts().sort_index().plot(kind='bar',ax=axes[1,1],figsize=(10, 8), color='blue', alpha=0.5, title='Frecuencia de entrenamiento', 
                                                                     xlabel='Días de entrenamiento por semana')
axes[0, 0].set_xticklabels(axes[0, 0].get_xticklabels(), rotation=45)
axes[0, 1].set_xticklabels(axes[0, 1].get_xticklabels(), rotation=45)
axes[1, 0].set_xticklabels(axes[1, 0].get_xticklabels(), rotation=45)
axes[1, 1].set_xticklabels(axes[1, 1].get_xticklabels(), rotation=0)
plt.tight_layout()
plt.show()

Comentarios de cada una de las gráficas:
- El primer gráfico: nos dice cuántas personas hay en cada uno de los tres niveles de experiencia en el entrenamiento, esto es: principiante, intermedio y avanzado. Se observa cómo hay muchos menos avanzados que principiantes e intermedios.

- El segundo gráfico: nos informa del tipo de entrenamiento que realizan las personas. Los 4 tipos están bastante parejos, no obstante, fuerza y cardio son los más preponderantes.
- El tercer gráfico: simplemente informa de la cantidad de hombres y de mujeres en la muestra. Se ve que hay más hombres que mujeres. 
- El cuarto gráfico: detalla los días por semana que entrena cada persona. Se ve que lo más común es entrenar 3 o 4 días por semana. 

## OUTLIERS

<div style="background-color: blue; padding: 10px; border-radius: 5px;">
    Una vez que nos hemos hecho a la idea de como se comportan nuestros datos, vamos a detectar los OUTLIERS de cada una de las columnas. Calculamos numéricamente el de todas las 
    variables puesto que necesitamos conocer bien los datos que usaremos para entrenar nuestro modelo. En adición, representaremos algunas variables con el gráfico de cajas y bigotes. 
</div>

El método elegido para calcular los outliers es *Hampel X84* que utiliza la mediana y la MAD (Mean Absolute Deviation). Se considerarán outliers los puntos alejados más de $1.4826 \times \theta\times MAD$ de la mediana. Fijamos arbitrariamente el valor de $\theta=3$.

In [ ]:
from scipy.stats import median_abs_deviation

for col in columnas_a_graficar:
    columna = df[col]
    mediana = columna.median()
    MAD = median_abs_deviation(columna)
    inferior = mediana -  1.4826*3*MAD
    superior = mediana + 1.4826*3*MAD
    filtro_outliers_sup = columna > superior
    filtro_outliers_inf = columna < inferior
    
    if columna[filtro_outliers_inf].empty and columna[filtro_outliers_sup].empty:
        print(f'La columna {col} no tiene outliers para \u03B8 = 3')
    elif columna[filtro_outliers_inf].empty and not columna[filtro_outliers_sup].empty:
        print(f'Los outliers superiores de la columna {col} son:', columna[filtro_outliers_sup].tolist())
    elif not columna[filtro_outliers_inf].empty and columna[filtro_outliers_sup].empty:
        print(f'Los outliers inferiores de la columna {col} son:', columna[filtro_outliers_inf].tolist())
    else:
        print(f'Los outliers inferiores de la columna {col} son:', columna[filtro_outliers_inf].tolist())
        print(f'Los outliers superiores de la columna {col} son:', columna[filtro_outliers_sup].tolist())



Observamos que muchas de las columnas no tienen outliers. Por otra parte, la columna 'BMI' presenta una cantidad bastante alta de outliers, quizás el valor de $\theta$ es muy restrictivo con esta variable. 

Ahora hacemos los gráficos de caja y bigotes de las variables que presentan outliers y de la variable 'Avg_BPM'. Dado que este tipo de gráfico usa el método del rango intercuártilico para calcular los outliers, puede que aparezcan más que en el caso de *Hampel X84*.

In [ ]:
#Graficos de caja y bigotes
fig, axes = plt.subplots(2,2,figsize=(12,10))

df['Weight (kg)'].plot(kind='box', ax=axes[0,0], figsize=(11,9), boxprops=dict(color='blue'), medianprops=dict(linewidth=2,color='orange'), title='Weight (kg)',
                       whiskerprops=dict(color='blue'),capprops=dict(color='blue'))

df['Avg_BPM'].plot(kind='box', ax=axes[0,1], boxprops=dict(color='blue'), medianprops=dict(linewidth=2,color='orange'), title='Avg BPM', whiskerprops=dict(color='blue'),capprops=dict(color='blue'))

df['Calories_Burned'].plot(kind='box', ax=axes[1,0],boxprops=dict(color='blue'), medianprops=dict(linewidth=2,color='orange'), title='Calories Burned',
                           whiskerprops=dict(color='blue'),capprops=dict(color='blue'))

df['BMI'].plot(kind='box', ax=axes[1,1],boxprops=dict(color='blue'), medianprops=dict(linewidth=2,color='orange'), title='BMI', whiskerprops=dict(color='blue'),capprops=dict(color='blue'))


plt.tight_layout()
plt.show()

Se puede ver que han aparecido más outliers de los que había antes. En el caso de 'Weight', están muy cercanos al límite superior por lo que no los trataremos. En el caso de 'Calories Burned', aunque algunos ya se alejan un poco tampoco haremos nada con ellos. Lo mismo con 'BMI' que sí presenta outliers bastante distantes, pero los dejamos puesto que es una medida secundaria dependiente de dos variables independientes (peso y altura) y no nos es de mucha utilidad.

El motivo de que no tratemos los outliers es que no parecen responder a medidas erróneas sino a casos extremos de sobrepeso en el caso 'Weight' y 'BMI' (son variables correlacionadas).
Los outliers en 'Calories Burned' son verosímiles. 

## TEST ESTADISTICOS


<div style="background-color: blue; padding: 10px; border-radius: 5px;">
Una vez estudiados los outliers, pasamos a realizar algunos TEST ESTADÍSTICOS sobre algunas variables. En nuestro DataFrame tenemos ciertas variables categóricas como 'Gender', 'Exp_Level', 'Workout frequency' y 'Workout_Type'. Vamos a ver qué comportamientos tienen las variables cuantitativas dependiendo de la categoría a la que pertenecen. Despúes veremos las relaciones entre las propias variables categóricas.
</div>

En primer lugar, haremos una prueba *t-student* con las categorías de 'Gender', véase Male/Female, sobre la variable 'Calories_Burned', con el objetivo de ver si la media de calorías quemadas es diferente dependiendo del género.

In [ ]:
import scipy.stats as stats

calorias_hombres = df[df['Gender'] == 'Male']['Calories_Burned']
calorias_mujeres = df[df['Gender'] == 'Female']['Calories_Burned']

# Prueba t-student
t_stat, p_value = stats.ttest_ind(calorias_hombres, calorias_mujeres, equal_var=True)

print(f"Estadístico t: {t_stat}")
print(f"Valor p: {p_value}")
alpha=0.05
if p_value < alpha:
    print(f'Rechazamos la hipótesis nula: hay diferencia significativa entre la media de calorías quemadas.')
else:
    print(f'No se puede rechazar la hipótesis nula: no hay diferencia significativa entre la media de calorías quemadas.')

Concluimos que la media de calorías quemadas sí difiere significativamente en función del género.

Ahora realizamos un test ANOVA sobre la variable 'Calories_Burned' dependiendo del tipo de entrenamiento para ver si todos los entrenamientos son iguales a la hora de quemar calorías o si hay alguno que no. La hipótesis nula ($H_0$) es que la media de calorías quemadas es igual para todos los entrenamientos. Veamos:

In [ ]:
calorias_fuerza=df[df['Workout_Type']=='Strength']['Calories_Burned']
calorias_cardio=df[df['Workout_Type']=='Cardio']['Calories_Burned']
calorias_yoga=df[df['Workout_Type']=='Yoga']['Calories_Burned']
calorias_hiit=df[df['Workout_Type']=='HIIT']['Calories_Burned']

f_stat, p_value = stats.f_oneway(calorias_fuerza, calorias_cardio, calorias_yoga, calorias_hiit)

print(f"Estadístico F: {f_stat}")
print(f"Valor p: {p_value}")
alpha=0.05
if p_value < alpha:
    print(f'Rechazamos la hipótesis nula: las medias son significativamente diferentes.')
else:
    print(f'No se puede rechazar la hipótesis nula: las medias no son significativamente diferentes.')

El valor alto de $F$ y el valor de $P>0.05$ nos impiden rechazar la hipótesis nula, por lo que no hay evidencia suficiente para determinar que la media de calorías quemadas difiera en ningún tipo de entrenamiento.

Ahora hacemos otro test ANOVA entre 'Calories_Burned' y 'Experience_Level', para ver si la experiencia influye en la quema de calorías.

In [ ]:
calorias_principiante=df[df['Experience_Level']=='Principiante']['Calories_Burned']
calorias_intermedio=df[df['Experience_Level']=='Intermedio']['Calories_Burned']
calorias_avanzado=df[df['Experience_Level']=='Avanzado']['Calories_Burned']

f_stat, p_value = stats.f_oneway(calorias_principiante, calorias_intermedio, calorias_avanzado)

print(f"Estadístico F: {f_stat}")
print(f"Valor p: {p_value}")
alpha=0.05
if p_value < alpha:
    print(f'Rechazamos la hipótesis nula: las medias son significativamente diferentes.')
else:
    print(f'No se puede rechazar la hipótesis nula: las medias no son significativamente diferentes.')

En este caso sí podemos rechazar la hipótesis nula y afirmar que la media de calorías quemadas sí que varía de forma significativa en función del nivel de experiencia. No obstante, no conocemos si hay dos grupos en que no varía, o si varía en los tres.

Para acabar con los test ANOVA, estudiamos si la frecuencia de entrenamiento semanal influye en las calorías quemadas por entenamiento. A priori no deberían estar relacionadas, pero veamos:

In [ ]:
calorias_dos_dias=df[df['Workout_Frequency (days/week)']=='2']['Calories_Burned']
calorias_tres_dias=df[df['Workout_Frequency (days/week)']=='3']['Calories_Burned']
calorias_cuatro_dias=df[df['Workout_Frequency (days/week)']=='4']['Calories_Burned']
calorias_cinco_dias=df[df['Workout_Frequency (days/week)']=='5']['Calories_Burned']

f_stat, p_value = stats.f_oneway(calorias_principiante, calorias_intermedio, calorias_avanzado)

print(f"Estadístico F: {f_stat}")
print(f"Valor p: {p_value}")
alpha=0.05
if p_value < alpha:
    print(f'Rechazamos la hipótesis nula: las medias son significativamente diferentes.')
else:
    print(f'No se puede rechazar la hipótesis nula: las medias no son significativamente diferentes.')

El resultado es que las calorías quemadas por entrenamiento sí varían significativamente en función de los días a la semana que se entrena. Luego veremos una posible explicación.

Ahora hacemos un test $\chi^2$ entre todas las variables categóricas para ver si hay una asociación significativa entre ellas o no. 

In [ ]:
columnas_categoricas=['Gender','Workout_Type','Workout_Frequency (days/week)','Experience_Level']
for i, col1 in enumerate(columnas_categoricas):
    for col2 in columnas_categoricas[i+1:]:
        tabla_contingencia = pd.crosstab(df[col1], df[col2])
        chi2_stat, p_value, dof, expected = stats.chi2_contingency(tabla_contingencia)
        
        print(f'\nComparando {col1} y {col2}: Estadístico chi-cuadrado: {chi2_stat}, Valor p: {p_value}')
        alpha = 0.05
        if p_value < alpha:
            print(f'Rechazamos la hipótesis nula: Existe una asociación significativa entre {col1} y {col2}.')
        else:
            print(f'No se puede rechazar la hipótesis nula: No se encontró asociación significativa entre {col1} y {col2}.')
        print('----------------------------------------------------------------------------------------------------------------------')

Como vemos, la única relación que muestra una asociación significativa es 'Frecuencia semanal de entrenamiento' y 'Nivel de experiencia'.

## CORRELACIONES ENTRE VARIABLES


<div style="background-color: blue; padding: 10px; border-radius: 5px;">
A continuación, pasamos a ver las CORRELACIONES entre algunas variables. 
</div>

Empezamos por ver la correlación entre 'Workout_Frequency' y 'Experience_Level' que hemos visto antes que presentan una relación entre ellas. 

In [ ]:
#Graficamos un mapa de calor
import seaborn as sns

tabla_contingencia_new = pd.crosstab(df['Workout_Frequency (days/week)'],df['Experience_Level'])
plt.figure(figsize=(10, 8))
sns.heatmap(tabla_contingencia_new, annot=True, cmap="YlGnBu", fmt='d')
plt.title("Heatmap de frecuencias entre Frecuencia de entrenamiento y Nivel de experiencia")
plt.xlabel("Nivel de experiencia")
plt.ylabel("Frecuencia de entrenamiento")
plt.show()

Este mapa de calor nos ilustra bastante bien que a mayor nivel de experiencia la frecuencia de entrenamiento (días/semna) es mayor. 

En la siguiente gráfica mostramos la correlación entre la duración de la sesión de entrenamiento y las calorías quemadas, separado por género. Se puede ver que la relación es lineal, tal y como cabría esperar.

In [ ]:
#Gráfico de lineas
plt.figure(figsize=(10, 6))
sns.lineplot(data=df, x='Session_Duration (hours)', y="Calories_Burned",hue="Gender")
plt.title('Calorías quemadas dependiendo la duración del entrenamiento')
plt.xlabel('Duración del entrenamiento')
plt.ylabel('Calorías quemadas')
plt.grid(True)
plt.show()

Ahora pasamos a mostrar varios gráficos de tipo 'violin' y 'enjambre' para grupos de tres variables, de las cuales dos son categóricas y una continua.

El primero muestra las calorías quemadas en función del tipo de entrenamiento y clasificadas por género.

In [ ]:
fig=plt.figure(figsize=(9,7))
sns.violinplot(data=df,x='Workout_Type',y='Calories_Burned',hue='Gender')
plt.xlabel('Tipo de entrenamiento')
plt.ylabel('Calorías quemadas')
plt.title('Calorías quemadas dependiendo del tipo de entrenamiento y clasificado por género')
plt.show()

Se observa que las calorías quemadas son más o menos las mismas para todos los tipos de entrenamiento excepto para el Yoga, que quema algo menos (esto ya lo habíamos estudiado y ahora lo vemos con claridad). En todos los casos, los hombres queman más que las mujeres. El hecho de que los 'violines' presenten grosor hasta las puntas nos indica que los datos están dispersos y no concentrados en torno a la media.

El siguiente gráfico nos muestra las calorías quemadas en función del nivel de experiencia, de nuevo dividido por género. 

In [ ]:
fig=plt.figure(figsize=(9,7))
sns.violinplot(data=df,x='Experience_Level',y='Calories_Burned',hue='Gender')
plt.xlabel('Nivel de experiencia')
plt.ylabel('Calorías quemadas')
plt.title('Calorías quemadas dependiendo del nivel de experiencia y clasificado por género')
plt.show()

Se aprecia como a mayor nivel de experiencia las calorías quemadas por entrenamiento son mayores (ya habíamos visto que había diferencia significativa al menos en un grupo). La diferencia entre hombres y mujeres solo se aprecia en el nivel avanzado. En este caso los 'violines' del nivel principiante presentan una gran dispersion de los valores, mientras que los niveles intermedio y avanzado están más concentrados en torno a la media.

El siguiente gráfico de tipo 'enjambre' muestra las calorías quemadas en función de la frecuencia de entrenamiento smeanal y el nivel de experiencia. Aquí veremos por qué antes hemos obtenido que el número de días a la semana que se entrena influye en las calorías quemadas por entrenamiento. 

In [ ]:
#Gráfico de enjambre
fig, ax = plt.subplots(figsize=(10,8))
sns.swarmplot(data=df, x="Workout_Frequency (days/week)", y="Calories_Burned",hue='Experience_Level')
plt.xlabel('Días de entrenamiento por semana')
plt.ylabel('Calorías quemadas')
plt.title('Calorías quemadas en función de la frecuencia semanal de entrenamiento y del nivel de experiencia')
plt.legend()
plt.show()

Podemos ver que los principiantes entrenan 2 o 3 días por semana; los intermedios, 3 o 4; y los avanzados 4 o 5. A parte, las calorías quemadas, como acabamos de ver, aumentan con el nivel de experiencia. Y por último, en respuesta a por qué las calorías quemadas están relacionadas con la frecuancia semanal de entrenamiento, se observa que los que tienen una mayor experiencia entrenan más días por semana, y también son los que más calorías queman por entrenamiento. Así pues, concluimos que no se queman más calorías (en un entrenamiento) por entrenar más días a la semana, sino por tener un nivel de experiencia mayor. 

Otra variable interesante a estudiar es el procentaje graso ('Fat_Percentage'). Vamos a crear dos gráficos de violines para observar su comportamiento.

In [ ]:
#Gráfico de violines
fig=plt.figure(figsize=(9,7))
sns.violinplot(data=df,x='Experience_Level',y='Fat_Percentage',hue='Gender')
plt.xlabel('Nivel de experiencia')
plt.ylabel('Porcentaje de grasa corporal')
plt.title('Porcentaje de grasa corporal dependiendo del nivel de experiencia y clasificado por género')
plt.show()

Este gráfico nos muestra claramente cómo los que tienen un nivel avanzado presentan un porcentaje de grasa considerablemente menor que los de niveles intermedio y principiante, además con una dispersión mucho menor. Como cabría esperar, el procentaje graso en las mujeres es superior al de los hombres en todos los grupos, respondiendo a la proporción biológica. También resulta llamativo que no haya diferencia perceptible entre los niveles intermedio y principiante en el porcentaje graso, lo que nos sugiere que alcanzar niveles bajos de grasa requiere de mucho tiempo. 

In [ ]:
#Gráfico de violines
fig=plt.figure(figsize=(9,7))
sns.violinplot(data=df,x='Workout_Type',y='Fat_Percentage',hue='Experience_Level')
plt.xlabel('Nivel de experiencia')
plt.ylabel('Porcentaje de grasa corporal')
plt.title('Porcentaje de grasa corporal dependiendo del nivel de experiencia y clasificado por género')
plt.show()

Este último gráfico es también muy significativo: en primer lugar vemos la ya mencionada diferencia entre los niveles de experiencia y el porcentaje graso (solo los avanzados presentan diferencias significativas), pero además, se observa cómo el entrenamiento 'HIIT' es el que mejor resultados da en cuanto a porcentaje graso.

## MODELO DE REGRESION

<div style="background-color: blue; padding: 10px; border-radius: 5px;">
Finalmente, vamos a entrenar un modelo de regresión lineal para predecir las calorías quemadas por entrenamiento en función de los parámetros que más influyen en ello.
</div>

Entrenamos el modelo de regresión lineal usando las variables: edad, altura, peso, nivel de experiencia, género y duración de la sesión, que hemos visto que son las que más influyen en la quema de calorías. 

In [ ]:
import statsmodels.api as sm
from sklearn.model_selection import train_test_split


#Entrenamiento del modelo de regresión
X = pd.get_dummies(df[['Age','Height (m)','Weight (kg)','Experience_Level','Gender', 'Session_Duration (hours)']], drop_first=True)
X = X.astype(float)
X = sm.add_constant(X)
y = df['Calories_Burned']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

modelo_calorias = sm.OLS(y_train, X_train).fit()  

Ahora que ya tenemos entrenado el modelo, lo testeamos y lo evaluamos:

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

#Predicciones del modelo
y_pred = modelo_calorias.predict(X_test)

#Evaluación del modelo
bias = np.mean(y_pred - y_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse_normalizado = rmse / (np.max(y_test) - np.min(y_test))

predicciones= pd.DataFrame(y_pred, columns=['Calorías quemadas predichas'])
print(f'BIAS: {bias}, MAE: {mae}, RMSE: {rmse}, RMSE normalizado: {rmse_normalizado}')
print(predicciones.head(5))

Vemos que el error cuadrático medio normalizado es bastante bajo, lo que nos indica que el modelo se ajusta bastante bien a la realidad. 

Obtenemos ahora la ecuación del modelo para ver cuales son los parámetros que más influyen.

In [ ]:
coeficientes = modelo_calorias.params
print(coeficientes)

Vemos que las variables que más influyen son: duración de la sesión, género y altura. 

Para finalizar, representamos los resultados. Elegimos las variables 'Height (m)' y 'Session Duration' ya que la variable 'Gender', aunque influye más que altura, no se representa bien al ser 0 o 1.

In [ ]:
import plotly.graph_objects as go


# Crear los puntos para la recta de regresión
rango_altura = np.linspace(df['Height (m)'].min(), df['Height (m)'].max(), 20)
rango_duracion = np.linspace(df['Session_Duration (hours)'].min(), df['Session_Duration (hours)'].max(), 20)
altura_grid, duracion_grid = np.meshgrid(rango_altura, rango_duracion)


# Extraer específicamente los coeficientes deseados
alpha = coeficientes['const']  
coef_altura = coeficientes['Height (m)']
coef_duracion = coeficientes['Session_Duration (hours)']  
calories_pred = alpha + coef_altura * altura_grid + coef_duracion * duracion_grid


# Crear la figura conp plotly
fig = go.Figure()

# Agregar puntos de datos originales
fig.add_trace(go.Scatter3d(
    x=df['Height (m)'],
    y=df['Session_Duration (hours)'],
    z=df['Calories_Burned'],
    mode='markers',
    marker=dict(size=5, color='blue', opacity=0.8),
    name="Datos originales"
))

# Agregar la superficie de la recta de regresión
fig.add_trace(go.Surface(
    x=rango_altura,
    y=rango_duracion,
    z=calories_pred,
    colorscale="viridis",
    opacity=0.5,
    name="Recta de regresión"
))

# Configurar el diseño del gráfico
fig.update_layout(
    title="Regresión Lineal en 3D",
    scene=dict(
        xaxis_title="x - Altura (m)",
        yaxis_title="y - Duración de la sesión (horas) ",
        zaxis_title="z- Calorías quemadas"

    ),
    width=900,  # Ancho de la figura
    height=700  # Altura de la figura
)

# Mostrar el gráfico
fig.show()

## CONCLUSIONES

Para terminar, se exponen las conlcusiones a las que se han llegado sobre la quema de calorías:

- El tipo de entrenamiento no influye en la quema de calorías

- El nivel de experiencia sí influye en la quema de calorías (a mayor experiencia mayor quema).
- La duración de la sesión de entrenamiento sí influye en la quema de calorías (a mayor duración mayor quema).
- Factores biológicos como el género y la altura, influyen de manera considerable en la quema de calorías. Sobre todo el género, donde  los hombres queman bastante más que las mujeres.
- El peso o la edad apenas influyen en la quema de calorías.

